In [ ]:
import os
import shutil
import sys
import subprocess

# Script to prepare ligands for QligFEP
# --------------------------------------
# Where should the .pdb files be located:
#   The original .pdb files should be in the folder provided as an argument to this script.
#   Example: /home/user/my_ligands/
#
# What this script does:
#   - Creates the folder '1.ligprep' in the current directory if it doesn't exist.
#   - Copies all .pdb files from the source folder into '1.ligprep'.
#   - Runs QligFEP's 'generate_opls.py' script to generate necessary files (.lib, .prm, .log).
#
# How to run it:
#   python preplig.py /path/to/pdb_files_folder
#
# Requirements:
#   - Python 3.x
#   - QligFEP installed and environment variable $qligfep set to the root folder of QligFEP
#   - The script 'generate_opls.py' must exist in $qligfep/scripts/
#   - Permissions to copy files and run subprocess commands from Python

def main():
    if len(sys.argv) != 2:
        print("Usage: python preplig.py /path/to/pdb_files_folder")
        sys.exit(1)

    pdb_folder = sys.argv[1]
    ligprep_folder = "1.ligprep"

    # Check that source folder exists
    if not os.path.isdir(pdb_folder):
        print(f"Error: the folder {pdb_folder} does not exist.")
        sys.exit(1)

    # Create 1.ligprep folder if it does not exist
    if not os.path.exists(ligprep_folder):
        os.makedirs(ligprep_folder)
        print(f"Folder '{ligprep_folder}' created.")
    else:
        print(f"Folder '{ligprep_folder}' already exists.")

    # Copy all .pdb files from pdb_folder to ligprep_folder
    pdb_files = [f for f in os.listdir(pdb_folder) if f.endswith(".pdb")]
    if not pdb_files:
        print(f"No .pdb files found in {pdb_folder}")
        sys.exit(1)

    for pdb in pdb_files:
        src = os.path.join(pdb_folder, pdb)
        dst = os.path.join(ligprep_folder, pdb)
        shutil.copy2(src, dst)
        print(f"Copied {pdb} to {ligprep_folder}")

    # Run QligFEP's generate_opls.py script
    qligfep_path = os.environ.get("qligfep")
    if not qligfep_path:
        print("Error: environment variable $qligfep is not defined.")
        sys.exit(1)

    generate_script = os.path.join(qligfep_path, "scripts", "generate_opls.py")

    if not os.path.isfile(generate_script):
        print(f"Error: the script {generate_script} was not found.")
        sys.exit(1)

    # Run the script to generate .lib, .prm, and .log files
    cmd = ["python", generate_script]
    try:
        print("Running generate_opls.py...")
        subprocess.run(cmd, check=True)
        print("Files generated successfully.")
    except subprocess.CalledProcessError:
        print("Error running generate_opls.py.")
        sys.exit(1)

if __name__ == "__main__":
    main()